Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [54]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [2]:
url = 'http://commondatastorage.googleapis.com/books1000/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [3]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for notMNIST_large. This may take a while. Please wait.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
Extracting data for notMNIST_small. This may take a while. Please wait.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [12]:
def display_one(folder, file_index=0):
    files = os.listdir(folder)
    to_display = os.path.join(folder, files[file_index])
    print('Displaying file %s' % to_display)
    display(Image(to_display))


display_one(test_folders[0])

Displaying file notMNIST_small/A/MDEtMDEtMDAudHRm.png


Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [5]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  image_index = 0
  print(folder)
  for image in os.listdir(folder):
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[image_index, :, :] = image_data
      image_index += 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling notMNIST_large/A.pickle.
notMNIST_large/A
Could not read: notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : cannot identify image file <open file 'notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png', mode 'rb' at 0x106fc26f0> - it's ok, skipping.
Could not read: notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : cannot identify image file <open file 'notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png', mode 'rb' at 0x107b26150> - it's ok, skipping.
Could not read: notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : cannot identify image file <open file 'notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png', mode 'rb' at 0x106fc26f0> - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.128476
Standard deviation: 0.425507
Pickling notMNIST_large/B.pickle.
notMNIST_large/B
Could not read: notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : cannot identify image file <open file 'notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png', mode 'r

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [30]:
def plot_one(file, img_index=0):
    with open(file, 'rb') as f:
        dataset = pickle.load(f)
        plt.imshow(dataset[img_index, :, :])
        plt.show()


plot_one('notMNIST_small/A.pickle')

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [36]:
def check_classes(folder):
    all_files = os.listdir(folder)
    pickles = filter(lambda x: x.endswith('.pickle'), all_files)
    results = dict()
    for pickled_file in pickles:
        with open(os.path.join(folder, pickled_file), 'rb') as f:
            dataset = pickle.load(f)
            num_classes = dataset.shape[0]
            results[pickled_file] = num_classes
    print(results)


print('Num classes in small:')
check_classes('notMNIST_small')

print('Num classes in large:')
check_classes('notMNIST_large')

Num classes in small:
{'C.pickle': 1873, 'J.pickle': 1872, 'E.pickle': 1873, 'G.pickle': 1872, 'D.pickle': 1873, 'F.pickle': 1872, 'A.pickle': 1872, 'H.pickle': 1872, 'I.pickle': 1872, 'B.pickle': 1873}
Num classes in large:
{'C.pickle': 52912, 'J.pickle': 52911, 'E.pickle': 52912, 'G.pickle': 52912, 'D.pickle': 52911, 'F.pickle': 52912, 'A.pickle': 52909, 'H.pickle': 52912, 'I.pickle': 52912, 'B.pickle': 52911}


Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [31]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [37]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

In [39]:
def print_assert(msg, bool):
    if bool:
        print ('Test passed: [{}]'.format(msg))
    else:
        print('Test failed: [{}]'.format(msg))


def test_randomize():
    dataset = np.array([[[1]], [[2]], [[3]]])
    labels = np.array([1,2,3])
    
    shuffled_dataset, shuffled_labels = randomize(dataset, labels)
    
    print_assert('sum total dataset unchanged', sum(dataset) == sum(shuffled_dataset))
    print_assert('sum total dataset unchanged', sum(labels) == sum(shuffled_labels))
    
    for idx in xrange(labels.shape[0]):
        print_assert('index {} matches'.format(idx), shuffled_labels[idx] == shuffled_dataset[idx])

    
test_randomize()

Test passed: [sum total dataset unchanged]
Test passed: [sum total dataset unchanged]
Test passed: [index 0 matches]
Test passed: [index 1 matches]
Test passed: [index 2 matches]


Finally, let's save the data for later reuse:

In [40]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [41]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800441


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [51]:
def get_array_hashes(full_array):
    unique = set()
    for idx in xrange(full_array.shape[0]):
        array_hash = hash(full_array[idx].tostring())
        unique.add(array_hash)
    return unique

    
def check_overlap():
    with open(pickle_file, 'rb') as f:
        data_dict = pickle.load(f)

    train_array_set = get_array_hashes(data_dict['train_dataset'])
    valid_array_set = get_array_hashes(data_dict['valid_dataset'])
    test_array_set = get_array_hashes(data_dict['test_dataset'])
    
    train_valid_overlap_set = train_array_set.intersection(valid_array_set)
    train_test_overlap_set = train_array_set.intersection(test_array_set)
    test_valid_overlap_set = test_array_set.intersection(valid_array_set)
    
    print('Training set size: {}'.format(data_dict['train_dataset'].shape[0]))
    print('Validation set size: {}'.format(data_dict['valid_dataset'].shape[0]))
    
    print('Train-Valid overlap set size: {}'.format(len(train_valid_overlap_set)))
    print('Train-Test overlap set size: {}'.format(len(train_test_overlap_set)))
    print('Test-Valid overlap set size: {}'.format(len(test_valid_overlap_set)))


check_overlap()

Training set size: 200000
Validation set size: 10000
Train-Valid overlap set size: 953
Train-Test overlap set size: 1153
Test-Valid overlap set size: 55


---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [72]:
def get_dataset():
    with open(pickle_file, 'rb') as f:
        data_dict = pickle.load(f)
    return data_dict


def get_training_set(data_dict, num_samples=50):
    data = data_dict['train_dataset'][0:num_samples,:,:]
    labels = data_dict['train_labels'][0:num_samples]
    return data, labels


def get_test_set(data_dict):
    data = data_dict['test_dataset']
    labels = data_dict['test_labels']
    return data, labels


def make_one_hot_encoder(labels):
    enc = OneHotEncoder()
    enc.fit([[x] for x in labels])
    return enc


def reshape_data(input_dataset):
    '''
    input_dataset is array of (n_samples, w, h)
    
    returns array of (n_samples, w * h)
    '''
    n_samples = input_dataset.shape[0]
    width = input_dataset.shape[1]
    height = input_dataset.shape[2]
    return np.reshape(input_dataset, (n_samples, width * height))


def get_model():
    return LogisticRegression(penalty='l2',
                              dual=False,
                              tol=0.0001,
                              C=1.0,
                              fit_intercept=True,
                              intercept_scaling=1,
                              class_weight=None,
                              random_state=None)


def plot_sample(x_data):
    img_data = np.reshape(x_data, (28, 28))
    plt.imshow(img_data)
    plt.show()


def classify():
    num_train_samples = 1000
    data_dict = get_dataset()
    train_data, train_labels = get_training_set(data_dict, num_train_samples)

    X = reshape_data(train_data)
    
    model = get_model()
    print('Training...')
    model.fit(X, train_labels)
    print('Done Training.')
    
    test_data, test_labels = get_test_set(data_dict)
    X_test = reshape_data(test_data)
    score = model.score(X_test, test_labels)
    print('Score: {}'.format(score))
    
    # look at some examples:
    sample_num = 2124
    X_one = X_test[sample_num, :]
    label_one = test_labels[sample_num]
    predicted = model.predict(X_one)
    
    print('Expected Label: {}'.format(label_one))
    print('Predicted Label: {}'.format(predicted))
    plot_sample(X_one)
    
    print('The End.')

classify()

Training...
Done Training.
Score: 0.8333
Expected Label: 3
Predicted Label: [3]
The End.
